## Kinga Florek, florek.kinga@gmail.com 
## Michał Worsowicz, michal.worsowicz@o2.pl

# **Importy i funkcje**

In [ ]:
import cv2
import keras
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.utils import to_categorical
from tensorflow.keras import models
from tensorflow.keras import layers
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from keras.wrappers.scikit_learn import KerasClassifier

SZ = 28
affine_flags = cv2.WARP_INVERSE_MAP|cv2.INTER_LINEAR

def deskew(img):
    m = cv2.moments(img)
    if abs(m['mu02']) < 1e-2:
        return img.copy()
    skew = m['mu11']/m['mu02']
    M = np.float32([[1, skew, -0.5*SZ*skew], [0, 1, 0]])
    img = cv2.warpAffine(img,M,(SZ, SZ),flags=affine_flags)
    return img

def showOpencvImage(image, isGray=False):
    fig = plt.figure(figsize=(6, 6))
    plt.imshow(image, cmap = 'gray')
    plt.show()

def openCVHOG(im):
    winSize = (20,20)
    blockSize = (10,10)
    blockStride = (5,5)
    cellSize = (10,10)
    nbins = 9
    derivAperture = 1
    winSigma = -1.
    histogramNormType = 0
    L2HysThreshold = 0.2
    gammaCorrection = 1
    nlevels = 64
    signedGradients = True

    hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,histogramNormType,L2HysThreshold,gammaCorrection,nlevels, signedGradients)
    descriptor = np.ravel(hog.compute(im))
    
    return descriptor

def create_model(input_shape=None):
  network = models.Sequential()
  network.add(layers.Dense(512, activation='relu', input_shape=input_shape))
  network.add(layers.Dense(10, activation='softmax'))
  network.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
  return network

# **Obrazy oryginalne -> Deskew -> HOG -> Klasyfikator** [Zadanie 2a, 3a]

## Przygotowanie zbiorów danych

In [ ]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data() 

# dla klasycznych klasyfikatorów 
im_list = [train_images[i] for i in range(0,train_images.shape[0])] + [test_images[i] for i in range(0,test_images.shape[0])]

train_deskewed = [deskew(im) for im in im_list[0:train_images.shape[0]]]
test_deskewed = [deskew(im) for im in im_list[train_images.shape[0]:]]

train_deskewed = np.asarray(train_deskewed)
test_deskewed = np.asarray(test_deskewed)

im_list = [train_deskewed[i] for i in range(0,train_deskewed.shape[0])] + [test_deskewed[i] for i in range(0,test_deskewed.shape[0])]

hogdata = [openCVHOG(im) for im in im_list]
imData = np.float32(hogdata).reshape(-1,81)

# dla sieci
train_images = [openCVHOG(im) for im in train_deskewed]
train_images = np.float32(train_images).reshape(-1,81)

test_images = [openCVHOG(im) for im in test_deskewed]
test_images = np.float32(test_images).reshape(-1,81)

train_images = train_images.reshape((60000, 81))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 81))
test_images = test_images.astype('float32') / 255

encoded_train_labels = to_categorical(train_labels)
encoded_test_labels = to_categorical(test_labels)

## SVM 

In [ ]:
print("SVM, \t Obrazy oryginalne -> Deskew -> HOG -> Klasyfikator \n")
grid_params_svm = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

model = GridSearchCV(SVC(),param_grid=grid_params_svm)
model.fit(imData[0:600,:],train_labels[0:600])
print("Best params: ", model.best_params_)
pred_labels = model.predict(imData[600:1000,:])

mask = pred_labels==train_labels[600:1000]
correct = np.count_nonzero(mask)
print("Accuracy:", correct*100.0/pred_labels.size)

cm = confusion_matrix(train_labels[600:1000], pred_labels)
print("\n", cm)
print("\n", classification_report(train_labels[600:1000], pred_labels))

SVM, 	 Obrazy oryginalne -> Deskew -> HOG -> Klasyfikator 

Best params:  {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
Accuracy: 94.25

 [[39  0  0  0  0  0  0  0  0  0]
 [ 0 37  0  0  0  0  0  0  0  0]
 [ 0  0 33  0  0  0  0  2  0  0]
 [ 0  0  0 34  0  0  0  0  0  0]
 [ 1  0  0  0 44  0  1  0  0  0]
 [ 0  0  0  1  0 38  0  1  0  1]
 [ 0  0  1  0  0  1 38  0  0  0]
 [ 0  0  2  3  1  0  0 49  0  0]
 [ 1  0  0  0  1  0  0  0 36  0]
 [ 4  0  0  0  0  0  0  2  0 29]]

               precision    recall  f1-score   support

           0       0.87      1.00      0.93        39
           1       1.00      1.00      1.00        37
           2       0.92      0.94      0.93        35
           3       0.89      1.00      0.94        34
           4       0.96      0.96      0.96        46
           5       0.97      0.93      0.95        41
           6       0.97      0.95      0.96        40
           7       0.91      0.89      0.90        55
           8       1.00      0.95      0.97

## Las losowy

In [ ]:
print("Las losowy, \t Obrazy oryginalne -> Deskew -> HOG -> Klasyfikator \n")
grid_params_rfc = {'max_depth': [3, 5, 10],
                  'min_samples_split': [2, 5, 10]}

rfc = GridSearchCV(RandomForestClassifier(),param_grid=grid_params_rfc)
rfc = rfc.fit(imData[0:600,:],train_labels[0:600])
print("Best params: ", rfc.best_params_)
pred_labels = rfc.predict(imData[600:1000,:])

mask = pred_labels==train_labels[600:1000]
correct = np.count_nonzero(mask)
print("Accuracy:", correct*100.0/pred_labels.size)

cm = confusion_matrix(train_labels[600:1000], pred_labels)
print("\n", cm)
print("\n", classification_report(train_labels[600:1000], pred_labels))

Las losowy, 	 Obrazy oryginalne -> Deskew -> HOG -> Klasyfikator 

Best params:  {'max_depth': 10, 'min_samples_split': 5}
Accuracy: 93.0

 [[39  0  0  0  0  0  0  0  0  0]
 [ 0 37  0  0  0  0  0  0  0  0]
 [ 0  0 31  0  1  1  0  1  0  1]
 [ 0  0  1 33  0  0  0  0  0  0]
 [ 1  0  0  0 42  1  2  0  0  0]
 [ 0  0  1  0  0 38  0  0  0  2]
 [ 2  1  0  0  1  0 36  0  0  0]
 [ 0  0  1  2  1  0  0 51  0  0]
 [ 1  0  0  0  1  0  0  0 36  0]
 [ 2  0  0  0  0  2  1  1  0 29]]

               precision    recall  f1-score   support

           0       0.87      1.00      0.93        39
           1       0.97      1.00      0.99        37
           2       0.91      0.89      0.90        35
           3       0.94      0.97      0.96        34
           4       0.91      0.91      0.91        46
           5       0.90      0.93      0.92        41
           6       0.92      0.90      0.91        40
           7       0.96      0.93      0.94        55
           8       1.00      0.95      0

## Sieć

In [ ]:
print("Sieć, \t Obrazy oryginalne -> Deskew -> HOG -> Klasyfikator \n")

grid_params_network = {
    'batch_size' : [10, 20, 40],
    'epochs' : [1, 5, 10]
}

model = KerasClassifier(build_fn=create_model, verbose=0, input_shape=(81,))
network = GridSearchCV(estimator=model, param_grid=grid_params_network, n_jobs=-1, cv=3)
network = network.fit(train_images, encoded_train_labels)
print("Best params: ", network.best_params_)

pred_probabilities = network.predict(test_images)
pred_labels = np.argmax(pred_probabilities,-1)

test_acc = accuracy_score(test_labels, pred_probabilities)
print('Accuracy:', test_acc) 

cm = confusion_matrix(test_labels, pred_probabilities)
print("\n", cm)
print("\n", classification_report(test_labels, pred_probabilities))

Sieć, 	 Obrazy oryginalne -> Deskew -> HOG -> Klasyfikator 

Best params:  {'batch_size': 10, 'epochs': 10}


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Accuracy: 0.9558

 [[ 949    0    4    0    1    5    8    1    6    6]
 [   1 1122    2    1    3    0    0    3    2    1]
 [   0    2  986   11    0    0    1   20   10    2]
 [   0    1   11  974    0   10    0    9    5    0]
 [   2    2    4    0  938    2   12    0   12   10]
 [   1    0    1    3    0  873    2    1    9    2]
 [   6    3    0    0    7    5  933    0    4    0]
 [   3    4   41   35    0    0    0  932    5    8]
 [   0    0    3   10    7    4    2    3  934   11]
 [   7    1    6    3    4   15    4   13   39  917]]

               precision    recall  f1-score   support

           0       0.98      0.97      0.97       980
           1       0.99      0.99      0.99      1135
           2       0.93      0.96      0.94      1032
           3       0.94      0.96      0.95      1010
           4       0.98      0.96      0.97       982
           5       0.96      0.98      0.97       892
           6       0.97      0.97      0.97       958
           7   

# **Obrazy oryginalne -> HOG -> Klasyfikator** [Zadanie 2b]

## Przygotowanie danych

In [ ]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data() 

# dla klasycznych  klasyfikatorów 
im_list = [train_images[i] for i in range(0,train_images.shape[0])] + [test_images[i] for i in range(0,test_images.shape[0])]
hogdata = [openCVHOG(im) for im in im_list]
imData = np.float32(hogdata).reshape(-1,81)

# dla sieci 
train_images = [openCVHOG(im) for im in train_images]
train_images = np.float32(train_images).reshape(-1,81)

test_images = [openCVHOG(im) for im in test_images]
test_images = np.float32(test_images).reshape(-1,81)

train_images = train_images.reshape((60000, 81))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 81))
test_images = test_images.astype('float32') / 255

encoded_train_labels = to_categorical(train_labels)
encoded_test_labels = to_categorical(test_labels)

## SVM

In [ ]:
print("SVM, \t Obrazy oryginalne -> HOG -> Klasyfikator \n")

grid_params_svm = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

model = GridSearchCV(SVC(),param_grid=grid_params_svm)
model.fit(imData[0:600,:],train_labels[0:600])
print("Best params: ", model.best_params_)
pred_labels = model.predict(imData[600:1000,:])

mask = pred_labels==train_labels[600:1000]
correct = np.count_nonzero(mask)
print("Accuracy:", correct*100.0/pred_labels.size)

cm = confusion_matrix(train_labels[600:1000], pred_labels)
print("\n", cm)
print("\n", classification_report(train_labels[600:1000], pred_labels))

SVM, 	 Obrazy oryginalne -> HOG -> Klasyfikator 

Best params:  {'C': 1, 'kernel': 'linear'}
Accuracy: 90.0

 [[38  0  1  0  0  0  0  0  0  0]
 [ 0 37  0  0  0  0  0  0  0  0]
 [ 0  0 30  0  0  0  0  4  1  0]
 [ 0  0  0 30  0  2  0  1  0  1]
 [ 0  0  0  0 39  1  4  0  1  1]
 [ 0  0  2  1  0 36  0  0  1  1]
 [ 1  1  0  0  0  0 37  0  1  0]
 [ 0  0  4  1  1  0  0 49  0  0]
 [ 1  0  0  0  1  1  0  0 34  1]
 [ 3  0  0  0  0  0  1  1  0 30]]

               precision    recall  f1-score   support

           0       0.88      0.97      0.93        39
           1       0.97      1.00      0.99        37
           2       0.81      0.86      0.83        35
           3       0.94      0.88      0.91        34
           4       0.95      0.85      0.90        46
           5       0.90      0.88      0.89        41
           6       0.88      0.93      0.90        40
           7       0.89      0.89      0.89        55
           8       0.89      0.89      0.89        38
           9    

## Las losowy

In [ ]:
print("Las losowy, \t Obrazy oryginalne -> HOG -> Klasyfikator \n")

grid_params_rfc = {'max_depth': [3, 5, 10],
                  'min_samples_split': [2, 5, 10]}

rfc = GridSearchCV(RandomForestClassifier(),param_grid=grid_params_rfc)
rfc = rfc.fit(imData[0:600,:],train_labels[0:600])
print("Best params: ", rfc.best_params_)
pred_labels = rfc.predict(imData[600:1000,:])

mask = pred_labels==train_labels[600:1000]
correct = np.count_nonzero(mask)
print("Accuracy:", correct*100.0/pred_labels.size)

cm = confusion_matrix(train_labels[600:1000], pred_labels)
print("\n", cm)
print("\n", classification_report(train_labels[600:1000], pred_labels))

Las losowy, 	 Obrazy oryginalne -> HOG -> Klasyfikator 

Best params:  {'max_depth': 10, 'min_samples_split': 2}
Accuracy: 88.25

 [[35  4  0  0  0  0  0  0  0  0]
 [ 0 37  0  0  0  0  0  0  0  0]
 [ 0  0 32  0  0  0  0  2  0  1]
 [ 0  0  3 30  0  1  0  0  0  0]
 [ 1  0  0  0 42  0  2  0  0  1]
 [ 0  0  1  1  0 36  0  0  1  2]
 [ 2  1  0  0  1  1 35  0  0  0]
 [ 0  0  4  2  1  0  0 48  0  0]
 [ 1  2  0  0  1  2  0  0 30  2]
 [ 3  0  0  0  0  1  1  1  1 28]]

               precision    recall  f1-score   support

           0       0.83      0.90      0.86        39
           1       0.84      1.00      0.91        37
           2       0.80      0.91      0.85        35
           3       0.91      0.88      0.90        34
           4       0.93      0.91      0.92        46
           5       0.88      0.88      0.88        41
           6       0.92      0.88      0.90        40
           7       0.94      0.87      0.91        55
           8       0.94      0.79      0.86      

## Sieć


In [ ]:
print("Sieć, \t Obrazy oryginalne -> HOG -> Klasyfikator \n")

grid_params_network = {
    'batch_size' : [10, 20, 40],
    'epochs' : [1, 5, 10]
}

model = KerasClassifier(build_fn=create_model, verbose=0, input_shape=(81,))
network = GridSearchCV(estimator=model, param_grid=grid_params_network, n_jobs=-1, cv=3)
network = network.fit(train_images, encoded_train_labels)
print("Best params: ", network.best_params_)

pred_probabilities = network.predict(test_images)
pred_labels = np.argmax(pred_probabilities,-1)

test_acc = accuracy_score(test_labels, pred_probabilities)
print('Accuracy:', test_acc) 

cm = confusion_matrix(test_labels, pred_probabilities)
print("\n", cm)
print("\n", classification_report(test_labels, pred_probabilities))

Sieć, 	 Obrazy oryginalne -> HOG -> Klasyfikator 

Best params:  {'batch_size': 10, 'epochs': 10}


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Accuracy: 0.9389

 [[ 940    3    6    0    4    7   12    2    3    3]
 [   0 1124    2    1    4    0    1    0    2    1]
 [   2    4  962    5    1    1    0   33   20    4]
 [   0    0   17  954    0   11    0   12   14    2]
 [   6    3    0    0  944    1   12    0    4   12]
 [   3    1    1    8    2  809    1    2   48   17]
 [  16    3    0    0   16    5  910    0    7    1]
 [   5    5   57   16    1    1    0  906   11   26]
 [   3    1    9    6    5    7    5    6  904   28]
 [  17    1    0    3    3   11    2    7   29  936]]

               precision    recall  f1-score   support

           0       0.95      0.96      0.95       980
           1       0.98      0.99      0.99      1135
           2       0.91      0.93      0.92      1032
           3       0.96      0.94      0.95      1010
           4       0.96      0.96      0.96       982
           5       0.95      0.91      0.93       892
           6       0.97      0.95      0.96       958
           7   

# **Komentarz do zadania 2**

**SVM**

Dla danych bez wyrównania accuracy wynosi 90%, natomiast dla danych z wyrównaniem 94.25%.

**Lasy Losowe**

Dla danych bez wyrównania accuracy wynosi 88.25%, natomiast dla danych z wyrównaniem 93%.

**Sieci neuronowe**

Dla danych bez wyrównania accuracy wynosi 93.89%, natomiast dla danych z wyrównaniem 95.58%.

<br>

**Podsumowanie**

Możemy zauważyć, że accuracy się znacząco poprawia, jeżeli mamy dane po wyrównaniu. Najlepsze wyniki dają sieci neuronowe.

# **Obrazy oryginalne -> Deskew -> ReshapeTo1D -> Klasyfikator** [Zadanie 3b, 4a]

## Przygotowanie danych

In [ ]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data() 

# dla klasycznych klasyfikatorów
im_list = [train_images[i] for i in range(0,train_images.shape[0])] + [test_images[i] for i in range(0,test_images.shape[0])]

train_deskewed = [deskew(im) for im in im_list[0:train_images.shape[0]]]
test_deskewed = [deskew(im) for im in im_list[train_images.shape[0]:]]

train_deskewed = np.asarray(train_deskewed)
test_deskewed = np.asarray(test_deskewed)

train_deskewed = train_deskewed.reshape((-1, 28 * 28))
test_deskewed = test_deskewed.reshape((-1, 28 * 28))

im_list = [train_deskewed[i] for i in range(0,train_deskewed.shape[0])] + [test_deskewed[i] for i in range(0,test_deskewed.shape[0])]

imData = np.float32(im_list)

# dla sieci
train_images = train_deskewed.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_deskewed.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

encoded_train_labels = to_categorical(train_labels)
encoded_test_labels = to_categorical(test_labels)

## SVM

In [ ]:
print("SVM, \t Obrazy oryginalne -> Deskew -> ReshapeTo1D -> Klasyfikator \n")
grid_params_svm = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

model = GridSearchCV(SVC(),param_grid=grid_params_svm)
model.fit(imData[0:600],train_labels[0:600])
print("Best params: ", model.best_params_)
pred_labels = model.predict(imData[600:1000])

mask = pred_labels==train_labels[600:1000]
correct = np.count_nonzero(mask)
print("Accuracy:", correct*100.0/pred_labels.size)

cm = confusion_matrix(train_labels[600:1000], pred_labels)
print("\n", cm)
print("\n", classification_report(train_labels[600:1000], pred_labels))

SVM, 	 Obrazy oryginalne -> Deskew -> ReshapeTo1D -> Klasyfikator 

Best params:  {'C': 1, 'kernel': 'linear'}
Accuracy: 91.75

 [[38  0  0  0  0  1  0  0  0  0]
 [ 0 37  0  0  0  0  0  0  0  0]
 [ 0  0 33  0  0  0  0  0  1  1]
 [ 1  0  0 32  0  1  0  0  0  0]
 [ 0  0  1  0 42  1  0  0  0  2]
 [ 1  0  0  0  3 36  1  0  0  0]
 [ 1  0  1  0  0  0 37  0  1  0]
 [ 0  0  1  0  2  0  0 52  0  0]
 [ 0  1  1  2  0  1  0  0 33  0]
 [ 2  0  1  0  2  0  0  2  1 27]]

               precision    recall  f1-score   support

           0       0.88      0.97      0.93        39
           1       0.97      1.00      0.99        37
           2       0.87      0.94      0.90        35
           3       0.94      0.94      0.94        34
           4       0.86      0.91      0.88        46
           5       0.90      0.88      0.89        41
           6       0.97      0.93      0.95        40
           7       0.96      0.95      0.95        55
           8       0.92      0.87      0.89        

## Las losowy

In [ ]:
print("Las losowy, \t Obrazy oryginalne -> Deskew -> ReshapeTo1D -> Klasyfikator \n")
grid_params_rfc = {'max_depth': [3, 5, 10],
                  'min_samples_split': [2, 5, 10]}

rfc = GridSearchCV(RandomForestClassifier(),param_grid=grid_params_rfc)
rfc = rfc.fit(imData[0:600],train_labels[0:600])
print("Best params: ", rfc.best_params_)
pred_labels = rfc.predict(imData[600:1000])

mask = pred_labels==train_labels[600:1000]
correct = np.count_nonzero(mask)
print("Accuracy:", correct*100.0/pred_labels.size)

cm = confusion_matrix(train_labels[600:1000], pred_labels)
print("\n", cm)
print("\n", classification_report(train_labels[600:1000], pred_labels))

Las losowy, 	 Obrazy oryginalne -> Deskew -> ReshapeTo1D -> Klasyfikator 

Best params:  {'max_depth': 10, 'min_samples_split': 2}
Accuracy: 88.5

 [[36  0  0  0  0  1  1  0  1  0]
 [ 0 37  0  0  0  0  0  0  0  0]
 [ 0  1 33  0  0  0  0  0  0  1]
 [ 1  1  1 29  0  2  0  0  0  0]
 [ 0  1  1  0 40  1  1  0  0  2]
 [ 1  0  1  2  1 35  0  0  1  0]
 [ 1  0  1  0  0  0 37  0  1  0]
 [ 0  2  0  0  1  0  0 49  0  3]
 [ 0  1  0  3  0  4  0  0 30  0]
 [ 1  0  1  0  2  0  1  1  1 28]]

               precision    recall  f1-score   support

           0       0.90      0.92      0.91        39
           1       0.86      1.00      0.92        37
           2       0.87      0.94      0.90        35
           3       0.85      0.85      0.85        34
           4       0.91      0.87      0.89        46
           5       0.81      0.85      0.83        41
           6       0.93      0.93      0.93        40
           7       0.98      0.89      0.93        55
           8       0.88      0.7

## Sieć

In [ ]:
print("Sieć, \t Obrazy oryginalne -> Deskew -> ReshapeTo1D -> Klasyfikator \n")

grid_params_network = {
    'batch_size' : [10, 20, 40],
    'epochs' : [1, 5, 10]
}

model = KerasClassifier(build_fn=create_model, verbose=0, input_shape=(28*28,))
network = GridSearchCV(estimator=model, param_grid=grid_params_network, n_jobs=-1, cv=3)
network = network.fit(train_images, encoded_train_labels)
print("Best params: ", network.best_params_)

pred_probabilities = network.predict(test_images)
pred_labels = np.argmax(pred_probabilities,-1)

test_acc = accuracy_score(test_labels, pred_probabilities)
print('Accuracy:', test_acc) 

cm = confusion_matrix(test_labels, pred_probabilities)
print("\n", cm)
print("\n", classification_report(test_labels, pred_probabilities))

Sieć, 	 Obrazy oryginalne -> Deskew -> ReshapeTo1D -> Klasyfikator 



/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Best params:  {'batch_size': 40, 'epochs': 10}


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Accuracy: 0.985

 [[ 972    0    1    0    0    0    6    1    0    0]
 [   0 1128    1    2    0    2    1    1    0    0]
 [   1    1 1010    3    4    1    2    5    5    0]
 [   0    0    4  992    0    7    0    5    1    1]
 [   0    0    1    0  975    0    4    0    0    2]
 [   2    0    0    3    2  876    8    0    1    0]
 [   2    2    2    0    5    1  945    0    1    0]
 [   1    0    6    0    1    0    0 1018    0    2]
 [   5    0    1    3    2    2    1    2  956    2]
 [   0    1    1    0   15    2    0   12    0  978]]

               precision    recall  f1-score   support

           0       0.99      0.99      0.99       980
           1       1.00      0.99      1.00      1135
           2       0.98      0.98      0.98      1032
           3       0.99      0.98      0.99      1010
           4       0.97      0.99      0.98       982
           5       0.98      0.98      0.98       892
           6       0.98      0.99      0.98       958
           7    

# **Komentarz do zadania 3**

**SVM**

Dla danych surowych accuracy wynosi 91.75%, natomiast dla danych po OpenCVHOG 94.25%.

**Lasy Losowe**

Dla danych surowych accuracy wynosi 88.5%, natomiast dla danych po OpenCVHOG 93%.

**Sieci neuronowe**

Dla danych surowych accuracy wynosi 98.5%, natomiast dla danych po OpenCVHOG 95.58%.

<br>

**Podsumowanie**

Możemy zauważyć, że accuracy się znacząco poprawia, jeżeli mamy dane po OpenCVHOG, ale tylko dla klasycznych klasyfikatorów. W przypadku sieci neuronowych lepszy wynik jest dla danych surowych. W tym przypadku również najlepsze wyniki dają sieci neuronowe.

# **Obrazy oryginalne -> Deskew -> ReshapeTo1D -> Shuffle -> Klasyfikator** [Zadanie 4b]

## Przygotowanie danych

In [ ]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data() 

# dla klasycznych klasyfikatorów 
im_list = [train_images[i] for i in range(0,train_images.shape[0])] + [test_images[i] for i in range(0,test_images.shape[0])]

train_deskewed = [deskew(im) for im in im_list[0:train_images.shape[0]]]
test_deskewed = [deskew(im) for im in im_list[train_images.shape[0]:]]

train_deskewed = np.asarray(train_deskewed)
test_deskewed = np.asarray(test_deskewed)

train_deskewed = train_deskewed.reshape((-1, 28 * 28))
test_deskewed = test_deskewed.reshape((-1, 28 * 28))

np.random.shuffle(train_deskewed)
np.random.shuffle(test_deskewed)

im_list = [train_deskewed[i] for i in range(0,train_deskewed.shape[0])] + [test_deskewed[i] for i in range(0,test_deskewed.shape[0])]

imData = np.float32(im_list)

# dla sieci
train_images = train_deskewed.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_deskewed.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

encoded_train_labels = to_categorical(train_labels)
encoded_test_labels = to_categorical(test_labels)


## SVM

In [ ]:
print("SVM, \t Obrazy oryginalne -> Deskew -> ReshapeTo1D -> Shuffle -> Klasyfikator \n")
grid_params_svm = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

model = GridSearchCV(SVC(),param_grid=grid_params_svm)
model.fit(imData[0:600],train_labels[0:600])
print("Best params: ", model.best_params_)
pred_labels = model.predict(imData[600:1000])

mask = pred_labels==train_labels[600:1000]
correct = np.count_nonzero(mask)
print("Accuracy:", correct*100.0/pred_labels.size)

cm = confusion_matrix(train_labels[600:1000], pred_labels)
print("\n", cm)
print("\n", classification_report(train_labels[600:1000], pred_labels))

SVM, 	 Obrazy oryginalne -> Deskew -> ReshapeTo1D -> Shuffle -> Klasyfikator 

Best params:  {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
Accuracy: 9.25

 [[ 0 39  0  0  0  0  0  0  0  0]
 [ 0 37  0  0  0  0  0  0  0  0]
 [ 0 35  0  0  0  0  0  0  0  0]
 [ 0 34  0  0  0  0  0  0  0  0]
 [ 0 46  0  0  0  0  0  0  0  0]
 [ 0 41  0  0  0  0  0  0  0  0]
 [ 0 40  0  0  0  0  0  0  0  0]
 [ 0 55  0  0  0  0  0  0  0  0]
 [ 0 38  0  0  0  0  0  0  0  0]
 [ 0 35  0  0  0  0  0  0  0  0]]

               precision    recall  f1-score   support

           0       0.00      0.00      0.00        39
           1       0.09      1.00      0.17        37
           2       0.00      0.00      0.00        35
           3       0.00      0.00      0.00        34
           4       0.00      0.00      0.00        46
           5       0.00      0.00      0.00        41
           6       0.00      0.00      0.00        40
           7       0.00      0.00      0.00        55
           8       0.00     

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Las losowy

In [ ]:
print("Las losowy, \t Obrazy oryginalne -> Deskew -> ReshapeTo1D -> Shuffle -> Klasyfikator \n")
grid_params_rfc = {'max_depth': [3, 5, 10],
                  'min_samples_split': [2, 5, 10]}

rfc = GridSearchCV(RandomForestClassifier(),param_grid=grid_params_rfc)
rfc = rfc.fit(imData[0:600],train_labels[0:600])
print("Best params: ", rfc.best_params_)
pred_labels = rfc.predict(imData[600:1000])

mask = pred_labels==train_labels[600:1000]
correct = np.count_nonzero(mask)
print("Accuracy:", correct*100.0/pred_labels.size)

cm = confusion_matrix(train_labels[600:1000], pred_labels)
print("\n", cm)
print("\n", classification_report(train_labels[600:1000], pred_labels))

Las losowy, 	 Obrazy oryginalne -> Deskew -> ReshapeTo1D -> Shuffle -> Klasyfikator 

Best params:  {'max_depth': 5, 'min_samples_split': 10}
Accuracy: 8.25

 [[ 4 21  3  0  0  0  0  4  0  7]
 [ 1 21  7  0  0  0  1  6  0  1]
 [ 0 23  3  0  2  0  0  4  0  3]
 [ 2 21  4  0  0  0  0  4  1  2]
 [ 3 22  2  1  0  3  0  9  1  5]
 [ 1 24  3  0  0  0  0  8  0  5]
 [ 1 23  2  1  0  0  1  4  0  8]
 [ 3 33  4  3  2  0  1  3  0  6]
 [ 1 30  2  1  0  0  0  1  0  3]
 [ 2 27  1  0  1  0  0  3  0  1]]

               precision    recall  f1-score   support

           0       0.22      0.10      0.14        39
           1       0.09      0.57      0.15        37
           2       0.10      0.09      0.09        35
           3       0.00      0.00      0.00        34
           4       0.00      0.00      0.00        46
           5       0.00      0.00      0.00        41
           6       0.33      0.03      0.05        40
           7       0.07      0.05      0.06        55
           8       0.

## Sieć

In [ ]:
print("Sieć, \t Obrazy oryginalne -> Deskew -> ReshapeTo1D -> Shuffle -> Klasyfikator \n")

grid_params_network = {
    'batch_size' : [10, 20, 40],
    'epochs' : [1, 5, 10]
}

model = KerasClassifier(build_fn=create_model, verbose=0, input_shape=(28*28,))
network = GridSearchCV(estimator=model, param_grid=grid_params_network, n_jobs=-1, cv=3)
network = network.fit(train_images, encoded_train_labels)
print("Best params: ", network.best_params_)

pred_probabilities = network.predict(test_images)
pred_labels = np.argmax(pred_probabilities,-1)

test_acc = accuracy_score(test_labels, pred_probabilities)
print('Accuracy:', test_acc) 

cm = confusion_matrix(test_labels, pred_probabilities)
print("\n", cm)
print("\n", classification_report(test_labels, pred_probabilities))

Sieć, 	 Obrazy oryginalne -> Deskew -> ReshapeTo1D -> Shuffle -> Klasyfikator 



/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Best params:  {'batch_size': 10, 'epochs': 5}


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Accuracy: 0.1113

 [[  0 810  12  90  13   5   2  16  30   2]
 [  3 934  12 120   7  10   1  24  21   3]
 [  3 857   7  94  15   7   4  13  30   2]
 [  2 828  15 105  14   8   1  10  24   3]
 [  5 792  13 122   6   2   3  11  25   3]
 [  2 731   9  83  10   3   2  21  30   1]
 [  3 769  14 101   9   7   2  15  38   0]
 [  3 841   8 109  13  11   3  19  20   1]
 [  1 797   5  96  11   9   1  20  34   0]
 [  4 810  13 117  13   5   3  16  25   3]]

               precision    recall  f1-score   support

           0       0.00      0.00      0.00       980
           1       0.11      0.82      0.20      1135
           2       0.06      0.01      0.01      1032
           3       0.10      0.10      0.10      1010
           4       0.05      0.01      0.01       982
           5       0.04      0.00      0.01       892
           6       0.09      0.00      0.00       958
           7       0.12      0.02      0.03      1028
           8       0.12      0.03      0.05       974
       

# **Komentarz do zadania 4**

**SVM**

Dla danych oryginalnych accuracy wynosi 91.75%, natomiast dla danych "zmieszanych" 9.25%.

**Lasy Losowe**

Dla danych oryginalnych accuracy wynosi 88.5%, natomiast dla danych "zmieszanych" 8.25%.

**Sieci neuronowe**

Dla danych oryginalnych accuracy wynosi 98.5%, natomiast dla danych "zmieszanych" 11.13%.

<br>

**Podsumowanie**

Możemy zauważyć, że dla danych "zmieszanych" accuracy jest bardzo niskie i wynosi średnio około 10%.  